In [167]:
from llama_index.core import SimpleDirectoryReader

documentos = SimpleDirectoryReader(input_dir='documents')

In [168]:
documentos.input_files

[PosixPath('/home/renan-peres/Área de trabalho/LLM/RAG/documents/serenatto_cafes_especiais.pdf')]

In [169]:
docs = documentos.load_data()

In [170]:
docs

[Document(id_='03a5f625-3f7b-4136-903d-7282241c739e', embedding=None, metadata={'page_label': '1', 'file_name': 'serenatto_cafes_especiais.pdf', 'file_path': '/home/renan-peres/Área de trabalho/LLM/RAG/documents/serenatto_cafes_especiais.pdf', 'file_type': 'application/pdf', 'file_size': 133957, 'creation_date': '2025-04-25', 'last_modified_date': '2025-04-25'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='Serenatto  –  Loja  online  de  cafés  especiais   Informações  gerais   P:  Quais  produtos  são  vendidos  no  site  da  Serenatto?  R:  A  Serenatto  é  uma  loja  online  especializada  na  venda  de  grãos  de  cafés  especiais, 

In [171]:
len(docs)

10

In [172]:
print(docs[0].get_metadata_str())

page_label: 1
file_name: serenatto_cafes_especiais.pdf
file_path: /home/renan-peres/Área de trabalho/LLM/RAG/documents/serenatto_cafes_especiais.pdf
file_type: application/pdf
file_size: 133957
creation_date: 2025-04-25
last_modified_date: 2025-04-25


In [173]:
docs[0].__dict__

{'id_': '03a5f625-3f7b-4136-903d-7282241c739e',
 'embedding': None,
 'metadata': {'page_label': '1',
  'file_name': 'serenatto_cafes_especiais.pdf',
  'file_path': '/home/renan-peres/Área de trabalho/LLM/RAG/documents/serenatto_cafes_especiais.pdf',
  'file_type': 'application/pdf',
  'file_size': 133957,
  'creation_date': '2025-04-25',
  'last_modified_date': '2025-04-25'},
 'excluded_embed_metadata_keys': ['file_name',
  'file_type',
  'file_size',
  'creation_date',
  'last_modified_date',
  'last_accessed_date'],
 'excluded_llm_metadata_keys': ['file_name',
  'file_type',
  'file_size',
  'creation_date',
  'last_modified_date',
  'last_accessed_date'],
 'relationships': {},
 'metadata_template': '{key}: {value}',
 'metadata_separator': '\n',
 'text_resource': MediaResource(embeddings=None, data=None, text='Serenatto  –  Loja  online  de  cafés  especiais   Informações  gerais   P:  Quais  produtos  são  vendidos  no  site  da  Serenatto?  R:  A  Serenatto  é  uma  loja  online  e

In [174]:
from llama_index.core.node_parser import SentenceSplitter

# tecnica chunks
# chunk size = numero de caracteres que serao usados para criar os blocos
node_parser = SentenceSplitter(chunk_size=1200)

In [175]:
nodes = node_parser.get_nodes_from_documents(docs, show_progress=True)

Parsing nodes: 100%|██████████| 10/10 [00:00<00:00, 115.09it/s]


In [176]:
len(nodes)

10

In [177]:
nodes[0]

TextNode(id_='53faf23d-561b-4495-a090-0bc5cb0a129c', embedding=None, metadata={'page_label': '1', 'file_name': 'serenatto_cafes_especiais.pdf', 'file_path': '/home/renan-peres/Área de trabalho/LLM/RAG/documents/serenatto_cafes_especiais.pdf', 'file_type': 'application/pdf', 'file_size': 133957, 'creation_date': '2025-04-25', 'last_modified_date': '2025-04-25'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='03a5f625-3f7b-4136-903d-7282241c739e', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '1', 'file_name': 'serenatto_cafes_especiais.pdf', 'file_path': '/home/renan-peres/Área de trabalho/LLM/RAG/documents/serenatto_cafes_especiais.pdf', 'file_type': 'application/pdf', 'file_size': 133957, 

In [178]:
# embedding cria vetores
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [179]:
# Classe personalizada para adptar a assinatura esperada pelo Chroma
class ChromaEmbeddingWrapper:
    def __init__(self, model_name):
        self.model = HuggingFaceEmbedding(model_name=model_name)

    def __call__(self, input):
        return self.model.embed(input)

In [180]:
embed_model_chroma = ChromaEmbeddingWrapper(model_name='intfloat/multilingual-e5-large')

In [181]:
import chromadb

db = chromadb.PersistentClient(path='./chroma_db')
chroma_client = db

collection_name = 'documentos_serenatto'

try:
    chroma_collection = chroma_client.get_or_create_collection(
        name = collection_name,
        embedding_function = embed_model_chroma 
    )

except Exception as e:
    print(f'Erro ao carregar ou criar coleção: {e}')

In [182]:
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext

In [183]:
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [184]:
embed_model = HuggingFaceEmbedding(model_name='intfloat/multilingual-e5-large')

In [185]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex(nodes, storage_context=storage_context, embed_model=embed_model)

In [186]:
from llama_index.core import load_index_from_storage

index = load_index_from_storage(storage_context, embed_model=embed_model)

In [187]:
from dotenv import load_dotenv
import os
load_dotenv()
GROQ_API = os.getenv('GROQ_API')

In [188]:
from llama_index.llms.groq import Groq

llm = Groq(model='llama3-70b-8192', api_key=GROQ_API)

In [189]:
query_engine = index.as_query_engine(llm=llm, similarity_top_k=2)

In [190]:
query_engine.query('Quais grãos estão disponíveis?')

Response(response='Os grãos disponíveis são: Bourbon vermelho, Bourbon amarelo, Blend especial (Mistura de Bourbon amarelo e vermelho), Catuaí amarelo, Geisha e Yirgacheffe.', source_nodes=[NodeWithScore(node=TextNode(id_='ece5cfe7-2300-449a-90ec-4a6fb4af8977', embedding=None, metadata={'page_label': '1', 'file_name': 'serenatto_cafes_especiais.pdf', 'file_path': '/home/renan-peres/Área de trabalho/LLM/RAG/documents/serenatto_cafes_especiais.pdf', 'file_type': 'application/pdf', 'file_size': 133957, 'creation_date': '2025-04-25', 'last_modified_date': '2025-04-25'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='25809342-cf96-45b0-b636-4ca19d6437ae', node_type='4', metadata={'page_label': '1', 'file_name':

In [191]:
query_engine.query('Quais grãos estão disponíveis?').response

'Os grãos disponíveis são: Bourbon vermelho, Bourbon amarelo, Blend especial (Mistura de Bourbon amarelo e vermelho), Catuaí amarelo, Geisha e Yirgacheffe.'

In [192]:
query_embedding = embed_model.get_text_embedding('Quais grãos estão disponíveis?')

In [193]:
chroma_collection.query(query_embedding, n_results=2, include=['distances', 'embeddings'])

{'ids': [['ece5cfe7-2300-449a-90ec-4a6fb4af8977',
   '16897708-dfee-4024-8b1c-b48efc2f8f58']],
 'embeddings': [array([[ 0.0133445 , -0.0119672 , -0.00394492, ..., -0.00065692,
          -0.05261652,  0.00293139],
         [ 0.0133445 , -0.0119672 , -0.00394492, ..., -0.00065692,
          -0.05261652,  0.00293139]], shape=(2, 1024))],
 'documents': None,
 'uris': None,
 'included': ['distances', 'embeddings'],
 'data': None,
 'metadatas': None,
 'distances': [[0.3484959602355957, 0.3484959602355957]]}

In [194]:
# adicionando um chat que transforma a entrada inicial e informação recuperada em um contexto e cria uma resposta mais elaborada
chat_engine = index.as_chat_engine(mode='context', llm=llm)

In [195]:
query = chat_engine.chat('Quais são os grãos disponíveis?').response
print(query)

Os grãos disponíveis são: Bourbon vermelho, Bourbon amarelo, Blend especial (Mistura de Bourbon amarelo e vermelho), Catuaí amarelo, Geisha e Yirgacheffe.


In [196]:
query = chat_engine.chat('Quais são os preços dos grãos?').response
print(query)

Os preços são os seguintes: Bourbon vermelho: R$ 41,00 
Bourbon amarelo: R$ 43,00 
Blend especial: R$ 37,50 
Catuaí amarelo: R$ 55,00 
Geisha: R$ 105,00 
Yirgacheffe: R$ 110,00


In [197]:
query = chat_engine.chat('Você pode me dar mais detalhes sobre o catuai amarelo?').response
print(query)

Este café possui doçura máxima (5/5) e corpo médio-alto (4/5) e amargor baixo (1/5). Ele ainda carrega uma expressiva acidez e notas maravilhosas de pitanga, fugindo muito do tradicional. É um café para quem busca uma experiência sensorial diferenciada. O processo é fermentado, cultivado em altitude de 1.200m, com torra média.


In [198]:
query = chat_engine.chat('Qual é o preço dele?').response
print(query)

R$ 55,00


In [199]:
chat_engine.chat_history

[ChatMessage(role=<MessageRole.USER: 'user'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='Quais são os grãos disponíveis?')]),
 ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={'tool_calls': [ChatCompletionMessageToolCall(id='call_fstk', function=Function(arguments='{"input":"Quais são os grãos disponíveis?"}', name='query_engine_tool'), type='function')]}, blocks=[]),
 ChatMessage(role=<MessageRole.TOOL: 'tool'>, additional_kwargs={'name': 'query_engine_tool', 'tool_call_id': 'call_fstk'}, blocks=[TextBlock(block_type='text', text='Os grãos disponíveis são: Bourbon vermelho, Bourbon amarelo, Blend especial (Mistura de Bourbon amarelo e vermelho), Catuaí amarelo, Geisha e Yirgacheffe.')]),
 ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='Os grãos disponíveis são: Bourbon vermelho, Bourbon amarelo, Blend especial (Mistura de Bourbon amarelo e vermelho), Catuaí amarelo, G

In [200]:
from llama_index.core.memory import ChatSummaryMemoryBuffer

In [201]:
memory = ChatSummaryMemoryBuffer(llm=llm, token_limit=256)

In [202]:
chat_engine = index.as_chat_engine(
    chat_mode='context',
    llm=llm,
    memory=memory,
    system_prompt=('''Você é especialista em cafés especiais do Serenatto, uma loja online que vende grãos de cafés torrados.
                   Sua função é tirar dúvidas de forma simpática e natural sobre os grãos disponíveis''')
)

In [203]:
query = chat_engine.chat('Olá').response
print(query)

Olá! Seja bem-vindo ao Serenatto, sua loja online de cafés especiais. Estou aqui para ajudar com qualquer dúvida sobre nossos cafés. O que você gostaria de saber?


In [204]:
query = chat_engine.chat('Você pode me dar mais detalhes sobre o catuai amarelo?').response
print(query)

O Catuaí amarelo é um dos nossos cafés mais especiais! Ele possui uma doçura máxima (5/5) e um corpo médio-alto (4/5), com um amargor baixo (1/5). Além disso, ele carrega uma expressiva acidez e notas maravilhosas de pitanga, o que o faz fugir muito do tradicional.

É um café para quem busca uma experiência sensorial diferenciada. O processo de produção é fermentado, cultivado em altitude de 1.200m, com torra média. Se você está procurando por um café que seja realmente único, o Catuaí amarelo é uma ótima escolha!


In [205]:
import gradio as gr

In [206]:
def converse_com_bot(message, chat_history):
    response = chat_engine.chat(message)

    if chat_history is None:
        chat_history = []

    chat_history.append({"role": "user", "content": message})
    chat_history.append({"role": "assistant", "content": response.response})

    return "", chat_history

def resetar_chat():
    chat_engine.reset()
    return []

In [207]:
with gr.Blocks() as demo:
    gr.Markdown('# Chatbot da Serenatto')
    chatbot = gr.Chatbot(type='messages')
    msg = gr.Textbox(label='Digite a sua mensagem')
    limpar = gr.Button('Limpar')

    msg.submit(converse_com_bot, [msg, chatbot], [msg, chatbot])
    limpar.click(resetar_chat, None, chatbot, queue=False)

demo.launch(debug=True)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.
